**Importing the libraries**

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

**Generating dataset as Cosine function**

In [3]:
np.random.seed(0)
torch.manual_seed(0)

t = np.linspace(0,100,1000)
data = np.cos(t)      # cosine function

def create_sequences(data, seq_length):
  xs, ys = [],[]
  for i in range(len(data) - seq_length):
    x = data[i:i + seq_length]
    y = data[i + seq_length]
    xs.append(x)
    ys.append(y)

  return np.array(xs),np.array(ys)\

seq_length = 10
X, y = create_sequences(data, seq_length)

trainX = torch.tensor(X[:,:,None],dtype = torch.float32)
trainY = torch.tensor(y[:, None], dtype = torch.float32)

**Building the neural network** (RNN)

In [4]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.lstm = nn.LSTM(input_dim,hidden_dim, layer_dim, batch_first = True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x, h0 = None, c0 = None):
    if h0 is None or c0 is None:
      h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
      c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

    out, (hn ,cn) = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1 , :])
    return out, hn ,cn

**Training the module and testing its accuracy**

In [8]:
model = LSTMModel(input_dim = 1, hidden_dim = 100, layer_dim = 1, output_dim = 1)
criterion = nn .MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

num_epochs = 100
h0 ,c0 = None, None
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()

  outputs, h0, c0 = model(trainX,h0,c0)

  loss = criterion(outputs, trainY)
  loss.backward()
  optimizer.step()

  h0, c0 = h0.detach(), c0.detach()

  if (epoch + 1)% 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.1100
Epoch [20/100], Loss: 0.0448
Epoch [30/100], Loss: 0.0041
Epoch [40/100], Loss: 0.0007
Epoch [50/100], Loss: 0.0016
Epoch [60/100], Loss: 0.0005
Epoch [70/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0001
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
